In [13]:

from models.UNet2p5D.Unet import Unet2p5D

from datasets import Data
from load_config import load_config
from utils import model_from_config
from train import train

from torch import optim, nn
import torch
from torch.utils.data import DataLoader


In [14]:
config = load_config()

In [15]:
data = Data(config)

In [16]:
train_set = data.get_train()

In [17]:
val_set = data.get_val()

In [18]:
vol, seg = train_set[0]

In [19]:
vol.shape, seg.shape

(torch.Size([5, 512, 512]), torch.Size([5, 512, 512]))

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [21]:
net = model_from_config(config).to(device)
net.load_state_dict(torch.load('checkpoints/epoch1.ckpt'))

<All keys matched successfully>

In [22]:
batch_size = config['batch_size']
num_workers = 4
lr = 1e-4

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

weights = torch.tensor([0.1, 0.7, 3]).to(device)
loss_fn = nn.CrossEntropyLoss(weight=weights)

optimizer = optim.Adam(net.parameters(), lr=lr)

In [23]:
train(
    net=net,
    train_loader=train_loader,
    val_loader=val_loader,
    n_epochs=1,
    device=device,
    optimizer=optimizer,
    loss_fn=loss_fn,
    config=config
)

[Iter 1] loss: 0.3799785077571869 dice: {'empty': 0.9863142967224121, 'liver': 0.22829021513462067, 'cancer': 0.0391300730407238}


RuntimeError: CUDA out of memory. Tried to allocate 768.00 MiB (GPU 0; 10.76 GiB total capacity; 5.62 GiB already allocated; 567.88 MiB free; 7.38 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
torch.save(net.state_dict(), "./checkpoints/epoch1.ckpt")